In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA

In [3]:
def check_stationarity(series):
    adf_result = adfuller(series.dropna())
    kpss_result = kpss(series.dropna(), regression="c", nlags="auto")

    print(f"ADF p-value: {adf_result[1]}")  # < 0.05 → stationary
    print(f"KPSS p-value: {kpss_result[1]}")  # > 0.05 → stationary
df = pd.read_csv('D:/Documents/Bicocca/2Year/TimeSeries/project/data/ts2024.csv')
df['DateTime'] = pd.to_datetime(df['DateTime'], format='ISO8601')

hourly_index = pd.date_range(start=df['DateTime'].min(), end=df['DateTime'].max(), freq='H')

hourly_index = pd.date_range(start=df['DateTime'].min(), end=df['DateTime'].max(), freq='H')
df = df.set_index(hourly_index).rename_axis('date')

df['DayOfWeek'] = df.index.dayofweek


train = df[df['X'].notna()]  
test = df[df['X'].isna()]  

In [4]:


train2 = train[-3000:-1000]
test = train[-1000:]




In [6]:
import timesfm


ModuleNotFoundError: No module named 'timesfm'

In [ ]:
import timesfm

tfm = timesfm.TimesFm(
    context_len=<context>,
    horizon_len=<horizon>,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend=<backend>,
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")
